In [1]:
from __future__ import print_function
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from sympy.solvers import solve
from sympy import Symbol
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, Box
import ipywidgets as widgets

In [2]:
# Total population, N.
total_pop = 300000000
# Initial number of infected and recovered individuals, I0 and R0.
I0 = 1
# A grid of time points (in days)
t = np.linspace(0, 365, num=365)
style = {'description_width': 'initial'}
items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')

In [3]:
items= []

In [4]:
def dec_to_percentage_string(dec):
    rounded_str = str(round(dec * 100)) + '%'
    return rounded_str

In [5]:
def convert_mrr_to_days(mrr):
    x = Symbol('x')
    sol = solve(mrr * x - 1, x)
    sol_string = str(round(sol[0], 3))
    return sol_string

In [6]:
# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [7]:
def percentage_of(percentage, total):
    return round((percentage / 100) * total)

In [8]:
@interact(force_of_infection_rate=widgets.FloatSlider(
            value=0.2,
            min=0,
            max=0.6,
            step=0.01,
            description='Force of Infection Rate:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style = {'description_width': 'initial'}
        ),
        mean_recovery_rate=widgets.FloatSlider(
            value=0.1,
            min=0.001,
            max=0.2,
            step=0.005,
            description='Mean Recovery Time:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.3f',
            style = {'description_width': 'initial'}
        ),
        initial_immunized_percent=widgets.IntSlider(
            value=0,
            min=0,
            max=100,
            step=1,
            description='Percentage Initially Immunized:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style = {'description_width': 'initial',}
        ),
        total_pop=fixed(total_pop))
def plot_graph(total_pop, force_of_infection_rate, mean_recovery_rate, initial_immunized_percent=0):
    # Everyone else, S0, is susceptible to infection initially.
    initial_immunized_count = percentage_of(percentage=initial_immunized_percent, total=total_pop)
    S0 = total_pop - I0 - initial_immunized_count
    # Initial conditions vector
    y0 = S0, I0, initial_immunized_percent

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(total_pop, force_of_infection_rate, mean_recovery_rate))
    susceptible, infected, recovered = np.transpose(ret)

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig = plt.figure(facecolor='w', figsize=(15.0, 8.0))
    pop_formatted = f"{total_pop:,}"
    fig.suptitle(f"Population Size={pop_formatted}\n " \
        + f" Force of Infection Rate={dec_to_percentage_string(force_of_infection_rate)}\n" \
        + f" Mean Recovery Time={convert_mrr_to_days(mean_recovery_rate)} Days\n" \
        + f" Percent Initially Immunized={str(initial_immunized_percent)}")
    ax = fig.add_subplot(111, axisbelow=True)
    ax.plot(t, susceptible/300000000, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, infected/300000000, 'r', alpha=0.5, lw=2, label='Infected')
    ax.plot(t, recovered/300000000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax.set_xlabel('Time /days')
    ax.set_ylabel('Percent of population')
    ax.set_ylim(0,1.2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()

interactive(children=(FloatSlider(value=0.2, description='Force of Infection Rate:', max=0.6, step=0.01, style…

In [9]:
# plot_graph(total_pop, force_of_infection_rate=0.2, mean_recovery_rate=1./10)
# plot_graph(total_pop, force_of_infection_rate=0.3, mean_recovery_rate=1./10)
# plot_graph(total_pop, force_of_infection_rate=0.2, mean_recovery_rate=1./8)
# plot_graph(total_pop, contact_rate=0.2, mean_recovery_rate=1./5)

In [10]:
def f(x):
    return x

In [11]:
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…